# SPY Spreadsheet

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import itable

import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [5]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2020-06-16,315.64,307.67,315.48,312.96,137627500.00,311.59
2020-06-17,314.39,310.86,314.07,311.66,82954600.00,310.29
2020-06-18,312.30,309.51,310.01,311.78,80828700.00,310.41
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [6]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [7]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,26.20,26.07,26.20,26.18,1003200.00,26.18
1993-02-01,26.37,26.20,26.20,26.37,480500.00,26.37
1993-02-02,26.44,26.30,26.35,26.43,201300.00,26.43
1993-02-03,26.72,26.44,26.46,26.71,529400.00,26.71
1993-02-04,26.87,26.50,26.80,26.82,531500.00,26.82


Add technical indicator: 200 day MA

In [8]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [9]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2020-06-16,314.26,306.32,314.10,311.59,137627500.00,311.59,297.93
2020-06-17,313.01,309.50,312.69,310.29,82954600.00,310.29,298.05
2020-06-18,310.93,308.15,308.65,310.41,80828700.00,310.41,298.18
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64,298.28
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67,298.37


Add technical indicator: ATR

In [10]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [11]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr
date,,,,,,,,
2020-06-16,314.26,306.32,314.10,311.59,137627500.00,311.59,297.93,7.36
2020-06-17,313.01,309.50,312.69,310.29,82954600.00,310.29,298.05,7.09
2020-06-18,310.93,308.15,308.65,310.41,80828700.00,310.41,298.18,6.78
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64,298.28,6.86
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67,298.37,6.67


Add technical indicator: 5 day high, and 5 day low

In [12]:
high5 = pd.Series(ts.high).rolling(window=5).max()
low5 = pd.Series(ts.low).rolling(window=5).min()
ts['high5'] = high5
ts['low5'] = low5

In [13]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2020-06-16,314.26,306.32,314.10,311.59,137627500.00,311.59,297.93,7.36,320.98,295.44
2020-06-17,313.01,309.50,312.69,310.29,82954600.00,310.29,298.05,7.09,314.26,295.44
2020-06-18,310.93,308.15,308.65,310.41,80828700.00,310.41,298.18,6.78,314.26,295.44
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64,298.28,6.86,314.38,295.44
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67,298.37,6.67,314.38,306.32


Add technical indicator: RSI, and 2-period cumulative RSI

In [14]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.Series(ts.rsi2).rolling(window=2).sum()
ts['c2rsi2'] = c2rsi2

In [15]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2020-06-16,314.26,306.32,314.10,311.59,137627500.00,311.59,297.93,7.36,320.98,295.44,77.07,126.71
2020-06-17,313.01,309.50,312.69,310.29,82954600.00,310.29,298.05,7.09,314.26,295.44,62.17,139.24
2020-06-18,310.93,308.15,308.65,310.41,80828700.00,310.41,298.18,6.78,314.26,295.44,63.48,125.65
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64,298.28,6.86,314.38,295.44,31.38,94.85
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67,298.37,6.67,314.38,306.32,68.19,99.57


Add technical indicator: Midpoint

In [16]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [17]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2020-06-16,314.26,306.32,314.10,311.59,137627500.00,311.59,297.93,7.36,320.98,295.44,77.07,126.71,310.29
2020-06-17,313.01,309.50,312.69,310.29,82954600.00,310.29,298.05,7.09,314.26,295.44,62.17,139.24,311.26
2020-06-18,310.93,308.15,308.65,310.41,80828700.00,310.41,298.18,6.78,314.26,295.44,63.48,125.65,309.54
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64,298.28,6.86,314.38,295.44,31.38,94.85,310.46
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67,298.37,6.67,314.38,306.32,68.19,99.57,308.90


Add technical indicator: SMA10 of midpoint

In [18]:
sma10mp = pd.Series(ts.mp).rolling(window=10).mean()
ts['sma10mp'] = sma10mp

In [19]:
ts.head(10)

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1993-01-29,26.20,26.07,26.20,26.18,1003200.00,26.18,nan,nan,nan,nan,nan,nan,26.14,nan
1993-02-01,26.37,26.20,26.20,26.37,480500.00,26.37,nan,nan,nan,nan,nan,nan,26.29,nan
1993-02-02,26.44,26.30,26.35,26.43,201300.00,26.43,nan,nan,nan,nan,100.00,nan,26.37,nan
1993-02-03,26.72,26.44,26.46,26.71,529400.00,26.71,nan,nan,nan,nan,100.00,200.00,26.58,nan
1993-02-04,26.87,26.50,26.80,26.82,531500.00,26.82,nan,nan,26.87,26.07,100.00,200.00,26.69,nan
1993-02-05,26.85,26.65,26.80,26.80,492100.00,26.80,nan,nan,26.87,26.20,89.33,189.33,26.75,nan
1993-02-08,26.89,26.76,26.80,26.80,596100.00,26.80,nan,nan,26.89,26.30,89.33,178.66,26.83,nan
1993-02-09,26.71,26.56,26.71,26.61,122100.00,26.61,nan,nan,26.89,26.44,16.96,106.29,26.63,nan
1993-02-10,26.67,26.54,26.61,26.65,379600.00,26.65,nan,nan,26.89,26.50,37.28,54.25,26.60,nan


Add technical indicator: Standard Deviation

In [20]:
sd = pd.Series(ts.mp).rolling(window=10).std()
ts['sd'] = sd

In [21]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2020-06-16,314.26,306.32,314.10,311.59,137627500.00,311.59,297.93,7.36,320.98,295.44,77.07,126.71,310.29,311.54,7.30
2020-06-17,313.01,309.50,312.69,310.29,82954600.00,310.29,298.05,7.09,314.26,295.44,62.17,139.24,311.26,311.64,7.28
2020-06-18,310.93,308.15,308.65,310.41,80828700.00,310.41,298.18,6.78,314.26,295.44,63.48,125.65,309.54,311.63,7.29
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64,298.28,6.86,314.38,295.44,31.38,94.85,310.46,310.89,6.96
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67,298.37,6.67,314.38,306.32,68.19,99.57,308.90,309.77,6.16


In [22]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [23]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2020-06-16,314.26,306.32,314.10,311.59,137627500.00,311.59,297.93,7.36,320.98,295.44,77.07,126.71,310.29,311.54,7.30,326.13,296.95
2020-06-17,313.01,309.50,312.69,310.29,82954600.00,310.29,298.05,7.09,314.26,295.44,62.17,139.24,311.26,311.64,7.28,326.20,297.08
2020-06-18,310.93,308.15,308.65,310.41,80828700.00,310.41,298.18,6.78,314.26,295.44,63.48,125.65,309.54,311.63,7.29,326.20,297.05
2020-06-19,314.38,306.53,314.17,308.64,135239100.00,308.64,298.28,6.86,314.38,295.44,31.38,94.85,310.46,310.89,6.96,324.80,296.98
2020-06-22,311.05,306.75,307.99,310.67,74649389.00,310.67,298.37,6.67,314.38,306.32,68.19,99.57,308.90,309.77,6.16,322.09,297.44


Select a smaller time from for use with itable

In [24]:
df = ts['2019-01-01':]

In [25]:
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-01-02,244.03,238.92,238.95,243.03,126925200.00,243.03,263.74,6.02,244.21,227.08,84.93,168.27,241.47,239.55,5.10,249.76,229.35
2019-01-03,241.46,236.70,241.13,237.23,144140700.00,237.23,263.63,6.04,244.21,232.13,15.82,100.75,239.08,238.78,4.41,247.60,229.95
2019-01-04,245.87,240.10,240.51,245.17,142628800.00,245.17,263.56,6.23,245.87,236.70,73.94,89.76,242.99,238.51,4.01,246.53,230.49
2019-01-07,248.63,244.49,245.46,247.11,103139100.00,247.11,263.50,6.08,248.63,236.70,80.49,154.43,246.56,239.20,4.75,248.71,229.69
2019-01-08,249.95,246.74,249.48,249.43,102512600.00,249.43,263.49,5.87,249.95,236.70,87.84,168.33,248.34,240.25,5.52,251.29,229.21


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [26]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [27]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2019/01/02,244.03,238.92,238.95,243.03,126925200,243.03,263.74,6.02,244.21,227.08,84.93,168.27,241.47,239.55,5.10,249.76,229.35
2019/01/03,241.46,236.70,241.13,237.23,144140700,237.23,263.63,6.04,244.21,232.13,15.82,100.75,239.08,238.78,4.41,247.60,229.95
2019/01/04,245.87,240.10,240.51,245.17,142628800,245.17,263.56,6.23,245.87,236.70,73.94,89.76,242.99,238.51,4.01,246.53,230.49
2019/01/07,248.63,244.49,245.46,247.11,103139100,247.11,263.50,6.08,248.63,236.70,80.49,154.43,246.56,239.20,4.75,248.71,229.69
2019/01/08,249.95,246.74,249.48,249.43,102512600,249.43,263.49,5.87,249.95,236.70,87.84,168.33,248.34,240.25,5.52,251.29,229.21
2019/01/09,251.51,248.86,250.19,250.59,95006600,250.59,263.50,5.64,251.51,236.70,91.17,179.01,250.19,242.19,5.22,252.63,231.75
2019/01/10,251.75,248.19,248.93,251.48,96823900,251.48,263.49,5.49,251.75,240.10,93.77,184.94,249.97,243.88,4.65,253.18,234.57
2019/01/11,251.60,249.68,250.31,251.57,73858100,251.57,263.50,5.24,251.75,244.49,94.15,187.91,250.64,245.28,4.33,253.94,236.61
2019/01/14,250.91,249.08,249.51,250.04,70908200,250.04,263.51,5.04,251.75,246.74,32.27,126.42,250.00,246.10,4.38,254.85,237.34
2019/01/15,253.25,250.44,250.45,252.91,85208300,252.91,263.52,4.91,253.25,248.19,80.39,112.67,251.84,247.11,4.42,255.95,238.27
